In [1]:
from datasets import load_dataset
import random
import re
from tqdm import tqdm
import pickle
import pandas as pd
import torch
import numpy as np
from CBOW import CBOW
random.seed(42)


/root/TwoTowerSearch/week_2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
embedder = CBOW()
embedder.load_state_dict(torch.load("checkpoints/best.pt", weights_only=True))
embedder.eval()

CBOW(
  (embed): Embedding(76289, 128)
  (lin): Linear(in_features=128, out_features=76289, bias=True)
)

In [35]:
test_df = pd.read_csv("tokenized_marco/tokenized_marco_test.csv").iloc[:,1:]
#test_df.set_axis(['ID','Query','is_selected','Passages'], axis=1)

In [42]:
test_df.set_axis(['ID','query','is_selected',0,1,2,3,4,5,6,7,8,9,10],axis=1)

,ID,query,is_selected,0,1,2,3,4,5,6,7,8,9,10
0,0.0,[ 1559. 800. 359. 816. 11614.],[0. 0. 1. 0. 0. 0. 0.],[7.6000e+01 6.0800e+02 7.7600e+02 1.4660e+03 1...,[7.3210e+03 1.3870e+03 3.1180e+03 1.0000e+00 7...,[2.6890e+03 1.6400e+02 8.0000e+02 3.5900e+02 2...,[0.0000e+00 1.4400e+02 1.7300e+02 5.5380e+03 4...,[3.5900e+02 7.1300e+02 3.7000e+01 1.1614e+04 1...,[0.0000e+00 1.4400e+02 1.7300e+02 2.6000e+02 1...,[1.3308e+04 2.6685e+04 8.0000e+02 3.5900e+02 1...,0,0,0,0
1,1.0,[1.430e+02 6.400e+01 1.000e+00 2.588e+03 3.800...,[0. 1. 0. 0. 0. 0. 0. 0. 0.],[1.0000e+00 4.8310e+03 1.8070e+03 3.8000e+01 1...,[2.588e+03 3.800e+01 4.697e+03 4.698e+03 4.697...,[4.6970e+03 4.6980e+03 6.4000e+01 2.8400e+03 3...,[1.0220e+04 3.8000e+01 4.6970e+03 4.6980e+03 3...,[2.5880e+03 2.2000e+02 3.8000e+01 1.0000e+00 4...,[4.6970e+03 4.6980e+03 2.6000e+02 8.5140e+03 3...,[0.0000e+00 3.2690e+03 7.9900e+02 6.4100e+02 5...,[1.4400e+02 6.0800e+02 1.4891e+04 2.2080e+03 3...,[4.6970e+03 4.6980e+03 6.4000e+01 6.8800e+02 3...,0,0
2,2.0,[1.4300e+02 6.8000e+01 3.9000e+01 7.3547e+04],[0. 0. 0. 0. 0. 1. 0. 0. 0.],[7.3547e+04 1.0000e+00 7.3547e+04 3.8000e+01 3...,[0.0000e+00 1.0000e+00 7.3547e+04 3.8000e+01 3...,[7.3547e+04 3.8000e+01 3.9000e+01 5.1400e+02 1...,[1.2100e+02 0.0000e+00 1.0000e+00 2.3360e+03 6...,[1.0000e+00 7.3547e+04 6.8000e+01 2.6000e+02 1...,[7.3547e+04 3.8000e+01 3.9000e+01 1.4098e+04 1...,[1.0000e+01 2.1400e+02 5.0142e+04 4.1330e+03 1...,[1.5830e+03 3.9000e+01 1.0430e+03 1.0000e+00 7...,[7.3547e+04 6.8000e+01 3.9000e+01 3.7890e+03 8...,0,0
3,3.0,[2.0000e+00 9.5100e+02 1.0000e+01 1.1989e+04 1...,[0. 0. 0. 0. 0. 1. 0. 0. 0.],[1.1784e+04 1.4400e+02 6.0800e+02 3.7600e+02 0...,[2.5170e+03 6.5100e+02 7.4900e+02 2.6100e+02 3...,[0.0000e+00 3.9000e+01 5.4690e+03 4.2000e+01 1...,[4.9000e+01 3.2910e+03 7.6000e+01 6.4000e+01 1...,[4.3160e+03 1.1989e+04 1.5753e+04 1.0000e+01 1...,[0.0000e+00 1.0000e+00 2.0000e+00 1.5069e+04 6...,[0.0000e+00 3.9000e+01 0.0000e+00 4.3160e+03 0...,[2.8200e+02 2.9900e+02 6.4000e+01 1.4400e+02 3...,[4.3160e+03 1.1989e+04 1.5069e+04 5.4690e+03 4...,0,0
4,4.0,[ 143. 68. 39. 8629. 37. 39. 3541.],[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.],[8.6290e+03 6.8000e+01 2.6200e+02 3.5520e+03 8...,[3.5410e+03 8.6290e+03 4.2700e+02 5.9900e+02 4...,[3.5410e+03 8.6290e+03 6.8000e+01 1.0000e+00 5...,[3.5410e+03 8.6290e+03 6.8000e+01 1.0000e+00 3...,[3.5410e+03 8.6290e+03 1.9420e+03 8.6290e+03 6...,[6.2470e+03 8.6000e+01 4.9900e+04 8.6290e+03 6...,[5.7940e+03 3.9000e+01 3.5410e+03 7.1100e+02 4...,[3.5410e+03 8.6290e+03 6.8000e+01 4.2500e+02 5...,[3.2220e+03 0.0000e+00 3.5410e+03 8.6290e+03 6...,[3.541e+03 8.629e+03 1.998e+03 1.500e+01 1.000...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9645,9647.0,[6.200e+01 1.970e+02 1.730e+02 1.440e+02 1.078...,[0. 0. 0. 1. 0. 0. 0.],[1.9100e+02 8.5400e+02 3.9870e+03 1.5000e+01 2...,[5.3630e+03 6.4000e+01 5.3180e+03 2.9940e+03 3...,[5.3630e+03 6.4000e+01 5.3180e+03 2.9940e+03 3...,[5.3630e+03 6.4000e+01 5.3180e+03 2.9940e+03 3...,[6.8100e+02 4.0120e+03 2.3100e+02 4.2000e+01 1...,[1.4400e+02 1.7300e+02 5.5400e+02 8.0940e+03 1...,[0.0000e+00 3.5700e+02 8.3000e+02 8.3200e+02 4...,0,0,0,0
9646,9648.0,[ 143. 68. 701. 19486. 921. 37. 6...,[0. 0. 0. 0. 0. 0. 1. 0. 0.],[3.7000e+01 2.0900e+02 1.7680e+03 6.5000e+01 1...,[4.2600e+02 2.2220e+03 7.7690e+03 2.1400e+02 7...,[6.2470e+03 1.9486e+04 1.9486e+04 6.8000e+01 3...,[1.0000e+00 4.2600e+02 2.2220e+03 7.7690e+03 1...,[5.6020e+03 0.0000e+00 1.0400e+02 2.8490e+03 6...,[1.9486e+04 3.3670e+03 3.9000e+01 8.6500e+02 2...,[4.2600e+02 2.2220e+03 7.7690e+03 1.9486e+04 1...,[3.7300e+03 0.0000e+00 2.7227e+04 6.8000e+01 3...,[1.9486e+04 2.1570e+03 1.0000e+01 4.2600e+02 2...,0,0
9647,9649.0,[1.730e+02 2.140e+02 9.334e+03 1.771e+03 1.000...,[0. 0. 0. 0. 0. 1. 0. 0. 0.],[3.320e+02 3.900e+01 2.587e+04 6.800e+01 2.990...,[1.4650e+03 2.1400e+02 9.3340e+03 1.9624e+04 5...,[2.5700e+02 1.4400e+02 6.0950e+03 6.5000e+01 2...,[2.5870e+04 8.5700e+03 6.8000e+01 7.4900e+02 3...,[5.7810e+03 3.9000e+01